In [1]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
res_train = np.load('../data/preds/res_train_preds.npy', mmap_mode = 'r')
selu_train = np.load('../data/preds/selu_train_preds.npy', mmap_mode = 'r')
relu_train = np.load('../data/preds/relu_train_preds.npy', mmap_mode = 'r')
X_train = np.concatenate((res_train, selu_train, relu_train), axis = 1)
res_val = np.load('../data/preds/res_val_preds.npy', mmap_mode = 'r')
selu_val = np.load('../data/preds/selu_val_preds.npy', mmap_mode = 'r')
relu_val = np.load('../data/preds/relu_val_preds.npy', mmap_mode = 'r')
X_val = np.concatenate((res_val, selu_val, relu_val), axis = 1)
res_test = np.load('../data/preds/res_test_preds.npy', mmap_mode = 'r')
selu_test = np.load('../data/preds/selu_test_preds.npy', mmap_mode = 'r')
relu_test = np.load('../data/preds/relu_test_preds.npy', mmap_mode = 'r')
X_test = np.concatenate((res_test, selu_test, relu_test), axis = 1)

In [3]:
y_train = np.load('../data/y_train_k.npy', mmap_mode = 'r')
y_val = np.load('../data/y_val_k.npy', mmap_mode = 'r')
y_test = np.load('../data/y_test_k.npy', mmap_mode='r')

In [4]:
ins = Input(shape = (69,))
outs = Dense(23, activation = 'sigmoid')(ins)

model = Model(ins, outs)

In [5]:
opt = Adam(lr = .00002)
model.compile(opt, 'categorical_crossentropy', metrics = ['acc'])
stopper = EarlyStopping(monitor='val_acc', min_delta = .0002, patience = 2, mode='auto')
checker = ModelCheckpoint('stacker_test_best_4', monitor='val_acc', mode='auto', save_best_only = True)

In [6]:
model.fit(X_train, y_train,
          epochs=100, batch_size=32,
          validation_data=(X_val, y_val),
          callbacks = [stopper, checker])

Train on 3428511 samples, validate on 736105 samples
Epoch 1/100
3428511/3428511 [==============================] - 178s - loss: 0.8027 - acc: 0.9603 - val_loss: 0.0252 - val_acc: 0.9980
Epoch 2/100
3428511/3428511 [==============================] - 178s - loss: 0.0117 - acc: 0.9983 - val_loss: 0.0074 - val_acc: 0.9984
Epoch 3/100
3428511/3428511 [==============================] - 178s - loss: 0.0060 - acc: 0.9986 - val_loss: 0.0052 - val_acc: 0.9988
Epoch 4/100
3428511/3428511 [==============================] - 178s - loss: 0.0045 - acc: 0.9989 - val_loss: 0.0041 - val_acc: 0.9993
Epoch 5/100
3428511/3428511 [==============================] - 178s - loss: 0.0037 - acc: 0.9993 - val_loss: 0.0036 - val_acc: 0.9993
Epoch 6/100
3428511/3428511 [==============================] - 178s - loss: 0.0034 - acc: 0.9994 - val_loss: 0.0034 - val_acc: 0.9994
Epoch 7/100
3428511/3428511 [==============================] - 178s - loss: 0.0031 - acc: 0.9994 - val_loss: 0.0032 - val_acc: 0.9994
Epoch 8/1

KeyboardInterrupt: 

In [4]:
model = load_model('models/stacker_test_best_4')

In [5]:
from sklearn.metrics import accuracy_score

In [9]:
val_preds = model.predict(X_val)
int_preds_val = np.array([np.argmax(r) for r in val_preds])
int_y_val = np.array([np.argmax(r) for r in y_val])
val_acc = accuracy_score(int_y_val, int_preds_val)
print(val_acc)

0.999434863233


In [10]:
train_preds = model.predict(X_train)
int_preds_train = np.array([np.argmax(r) for r in train_preds])
int_y_train = np.array([np.argmax(r) for r in y_train])
train_acc = accuracy_score(int_y_train, int_preds_train)
print(train_acc)

0.999450490315


In [6]:
test_preds = model.predict(X_test)
int_preds = np.array([np.argmax(r) for r in test_preds])
int_y_test = np.array([np.argmax(r) for r in y_test])
test_acc = accuracy_score(int_y_test, int_preds)
print(test_acc)

0.998193005049


In [ ]:
np.save('../data/preds')